In [91]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Text Processing

    Read each sentence from training file and remove the tabs and extract id and tweet seperately. 

In [92]:
file = "data/train_tweets.txt"
temp = []
with open(file, 'r',encoding="utf-8") as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row) 

In [93]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
temp = []
elem = []

In [94]:
tw.head()

,User,Tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


In [95]:
tw.tail()

,User,Tweet
328927,4357,Steelbox Demonstrates Open Video Framework wit...
328928,4357,Small Businesses Rely on Sage to Help Them Rid...
328929,4357,TimeSight Systems™ Announces Next-Generation P...
328930,4357,Diebold Makes Its Leading Monitoring Solutions...
328931,4357,GVI Security Solutions to Introduce AutoIP™ VM...


In [96]:
tw.shape

(328932, 2)

### Statistics: can be seen that number of tweets per user on the lower side

In [97]:
cnt_user = tw['User'].value_counts()
cnt_user.describe()

count    9297.000000
mean       35.380445
std        28.146449
min         1.000000
25%        18.000000
50%        32.000000
75%        38.000000
max       284.000000
Name: User, dtype: float64

In [98]:
from collections import Counter
mostToptweeters= Counter(tw.User).most_common(9000)

mostToptweeters.sort()


In [99]:
mostToptweeters[:5]
c=dict(mostToptweeters)
key=list(c.keys())
value=list(c.values())
value.sort()


In [102]:
Filtered_Tweet=tw[tw.User.isin(key)]

bdTweet= Filtered_Tweet.groupby('User')
bdTweet=bdTweet.apply(lambda x: x if (len(x)<10) else x.sample(10).reset_index(drop=True))

bdTweet.rename(columns={ bdTweet.columns[0]: "Index" }, inplace = True)


In [103]:
Tweet=bdTweet.groupby(['User','Tweet'],as_index=False).sum()


In [104]:
Tweet.rename(columns={ Tweet.columns[1]: "User" }, inplace = True)


In [105]:
X = Tweet.Tweet
print(len(X))
y = Tweet.User
print(len(y))

89249
89249


In [106]:
Tweet.shape

(89249, 2)

In [107]:
Tweets=X
Users=y

In [108]:
#vectorizer = CountVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 2)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(Tweets)
print(vectors.shape)

(89249, 91360)


In [110]:
X_train, X_test, y_train, y_test = train_test_split(vectors,Users, test_size=0.2, random_state=1337)

In [111]:
%%time
svm = LinearSVC()
svm.fit(X_train, y_train)

MemoryError: 

In [112]:
predictions = svm.predict(X_test)
print(list(predictions[0:10]))

NotFittedError: This LinearSVC instance is not fitted yet

In [ ]:
y_test.shape

In [45]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.13362093114460194


In [58]:
X_train, X_test, y_train, y_test = train_test_split(vectors,Users, test_size=0.2, random_state=1337)

In [126]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=5)
vectors = vectorizer.fit_transform(Tweets)
print(vectors.shape)

(112429, 50051)


In [127]:
%%time
svm = LinearSVC()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)
print(accuracy_score(y_test, predictions))

0.22067063950902785
Wall time: 11min 5s


In [ ]:
%%time
lR = LogisticRegression()
lR.fit(X_train, y_train)
predictions = lR.predict(X_test)
print(accuracy_score(y_test, predictions))


In [ ]:
%%time
test_model(vectorizer, 'LR')

Features:  203300
FRows: 207052


In [29]:
def prepare_test_data(vec):
    file1 = "data/test_tweets_unlabeled.txt"
    with open(file1, 'r',encoding="utf-8") as data:
        temp = [line for line in data]    
    unlabel = pd.Series(temp)
    unlabel_dtm = vec.transform(unlabel)
    return unlabel_dtm
    
def submission_file(data):
    import csv
    with open('predicted.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(['Id','Predicted'])
        for count,predicted in enumerate(data):
            writer.writerow([count+1,predicted])

In [30]:
unlabel_dtm = prepare_test_data(vectorizer) 
unlabel_pred = svm.predict(unlabel_dtm)
submission_file(unlabel_pred)

In [18]:
filename="LogisticRegression.sav"
pickle.dump(lr,open(filename,'wb'),protocol=4)
